# Are-MipiA1A2 : Venin

In [1]:
import numpy as np
from numpy import random
import matplotlib.pyplot as plt


anti_coagulantefficacite = 0.1
globule_blancefficacite = 0
veninefficacite = 0.1
plaquettescoagulation = 2
veninechange=0.5
globule_blancechange=0.5

In [2]:
def init_matrice(N):
    """Int -> Mat[Cell]
    crée une matrice de taille N*N de type : Mat[list[Gr,Gb,Pl,Ve,Co,Ac,boost]]"""
    matrix = np.zeros((N,N,7))
    mat = np.random.uniform(low=0.878, high=0.985, size=(N,N))
    for i in range(N):
        for j in range(N):
            x = mat[i,j]
            matrix[i,j]=(x , (1 - x)*0.023 , (1 - x)*0.977,0,(1 - x)*0.977,0,1)
    return matrix
            

In [3]:
def init_matrice_venin(N,x,y,Ve):
    matrix = np.zeros((N,N,7))
    mat = np.random.uniform(low=0.878, high=0.985, size=(N,N))
    for i in range(N):
        for j in range(N):
            a = mat[i,j]
            matrix[i,j]=(a , (1 - x)*0.023 , (1 - a)*0.977,0,(1 - a)*0.977,0,1)
    matrix[x,y,3]=Ve
    return matrix

In [14]:
def randomlist(N):
    """Int => list[int]*
    retourne une liste de nombres dans un ordre aléatoire len(list)=N """
    list0 = [i for i in range(N)]
    list1=[]
    rlist=[]
    for i in range(N):
        e = np.random.choice(len(list0))
        rlist.append(list0[e])
        list1=[j for j in list0 if j!=list0[e]]
        list0=list1
    return rlist

In [5]:
def selfinteract(mat,N):
    matrix = mat
    a = [0 for i in range(7)]
    for i in range(N):
        for j in range(N):
            
            a = mat[i,j]
            
            #globule_rouge+venin
            if a[3] > 0 and a[0] > 0:
                a[0] = a[0] - a[0]*veninefficacite
                if a[0]<= 0 : 
                    a[0] = 0.0000001
            #globule_blanc+venin
            if a[3] > 0:
                a[3] = a[3] - a[1]*globule_blancefficacite
                a[1] = 0
                a[6] = 10
                if a[3] < 0 : 
                    a[1]=-a[3]/globule_blancefficacite
                    a[3] = 0
            #plaquettes+plaquettes
            if a[2]>0:
                a[4] = a[4] + a[2]*plaquettescoagulation*1/a[0]
                a[2] = a[2] - a[2]/plaquettescoagulation*1/a[0]
                if a[2]<0 : 
                    a[4]= a[4]+a[2]*plaquettescoagulation 
                    a[2]=0
            #coagulation+anti_coagulant
            if a[5]>0:
                a[4] = a[4] - a[5]*anti_coagulantefficacite
                a[5] = a[5] - a[5]/anti_coagulantefficacite
                if a[5]<0 :
                    a[4] = a[4] -a[5]*anticoagulantefficacite
                    a[5] = 0
            #globule_blanc+coagulation
            if a[4]>0 and a[1]>0:
                a[4] = a[4] - a[1]*globule_blancefficacite
                if a[4]<0 :
                    a[4] = 0
            #venin+anti_coagulant
            if a[3] > 0 and a[5]>0:
                a[3] = a[3] - a[5]*anti_coagulantefficacite
                a[5] = a[5] - a[5]/anti_coagulantefficacite
                if a[3] < 0 :
                    a[5] = -a[3] + a[5]/anti_coagulantefficacite
            
            #veninaction
            #if a[3]>0:
            #    a[]
            
            matrix[i,j] = a
    return matrix
            

In [6]:
def interact(mat,i,j,k,l):
    matr = mat.copy()
    matr[i,j,3] = mat[i,j,3] + mat[k,l,3]*veninechange - mat[i,j,3]*veninechange
    matr[k,l,3] = mat[k,l,3] + mat[i,j,3]*veninechange - mat[k,l,3]*veninechange
    
    matr[i,j,1] = mat[i,j,1] + mat[k,l,1]*globule_blancechange
    matr[k,l,1] = mat[k,l,1] + mat[i,j,1]*globule_blancechange
    return matr

In [7]:
def inter_1(N,mat):
    
    
    for i in range(N):
        for j in range(N):
            a = mat[i,j]
            if a[3] > 0 :
                k = np.random.choice(4)
                if k == 0 and i > 0:
                    return interact(mat,i,j,i-1,j)
                if k == 1 and j < N:
                    return interact(mat,i,j,i,j+1)
                if k == 2 and i < N :
                    return interact(mat,i,j,i+1,j)
                if k == 3 and j > 0:
                    return interact(mat,i,j,i,j-1)
            
    return mat

In [ ]:
def inter_2(N,mat):
    a=randomlist(N)
    b=randomlist(N)
    for i in a:
        for j in n:
            

In [8]:
def lis_co(lis,N):
    """list[mat(cell)]*int => list(mat[float])
    convertie une liste de matrice de cellule en liste de matrice de float (Coagulation)"""
    long=len(lis)
    Lr = np.zeros((long,N,N))
    for n in range(long):
        mat = lis[n]
        for i in range(N):
            for j in range(N):
                Lr[n,i,j] = mat[i,j,3]
                
    return Lr
                

In [9]:
def main(N,x,y,Ve,n):
    mat = init_matrice_venin(N,x,y,Ve)
    rlist=np.zeros((n,N,N,7))
    for i in range(n):
        mat = inter_1(N,mat)
        mat = selfinteract(mat,N)
        rlist[i,:,:,:]=mat
        
    return lis_co(rlist,N)
    

In [10]:
main(3,1,1,1,10)

array([[[0.       , 0.       , 0.       ],
        [0.5      , 0.5      , 0.       ],
        [0.       , 0.       , 0.       ]],

       [[0.       , 0.       , 0.       ],
        [0.5      , 0.5      , 0.       ],
        [0.       , 0.       , 0.       ]],

       [[0.       , 0.       , 0.       ],
        [0.5      , 0.5      , 0.       ],
        [0.       , 0.       , 0.       ]],

       [[0.       , 0.       , 0.       ],
        [0.5      , 0.5      , 0.       ],
        [0.       , 0.       , 0.       ]],

       [[0.       , 0.       , 0.       ],
        [0.5      , 0.25     , 0.25     ],
        [0.       , 0.       , 0.       ]],

       [[0.       , 0.       , 0.       ],
        [0.375    , 0.375    , 0.25     ],
        [0.       , 0.       , 0.       ]],

       [[0.1875   , 0.       , 0.       ],
        [0.1875   , 0.375    , 0.25     ],
        [0.       , 0.       , 0.       ]],

       [[0.1875   , 0.       , 0.       ],
        [0.28125  , 0.28125  , 0.25     

In [11]:
0.234375  +  0.2578125 + 0.2578125 +  0.25  

1.0